In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

def jsonreader(exstring):
    ind_s = exstring.index("('")+2
    ind_e = exstring.index("')")
    json_data = exstring[ind_s:ind_e]
    return json.loads(json_data.encode('utf8').decode('unicode_escape'))

base_url = 'https://understat.com/match/'
matchnum = list(range(16376,16607)) 
#matches 16376->16607

urls = []

for each in matchnum:
    url = base_url+str(each)+'/'
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
    header = str(soup.find('title'))
    if header.find('Arsenal') != -1:
        urls.append(url)
        

print(urls)
# for each in urls:
#     r = requests.get(each)
#     soup = bs(r.content, 'html.parser')
#     if soup.find('title').__contains__('Arsenal'):
#         urls.remove(each)
    
# print(urls)


seasonrosterdata = []

['https://understat.com/match/16376/', 'https://understat.com/match/16394/', 'https://understat.com/match/16396/', 'https://understat.com/match/16407/', 'https://understat.com/match/16419/', 'https://understat.com/match/16434/', 'https://understat.com/match/16436/', 'https://understat.com/match/16446/', 'https://understat.com/match/16456/', 'https://understat.com/match/16468/', 'https://understat.com/match/16476/', 'https://understat.com/match/16489/', 'https://understat.com/match/16496/', 'https://understat.com/match/16512/', 'https://understat.com/match/16517/', 'https://understat.com/match/16526/', 'https://understat.com/match/16536/', 'https://understat.com/match/16548/', 'https://understat.com/match/16562/', 'https://understat.com/match/16576/']


In [2]:
r1 = requests.get('https://understat.com/match/16376')

soup1 = bs(r1.content, 'html.parser')

scripts1 = soup1.find_all('script')

rosters_json_string1 = ''
for each in scripts1:
    if 'rostersData' in each.text:
        rosters_json_string = each.text.strip()

json_dict1 = jsonreader(rosters_json_string)

# column and values names for an individual player
firstplayer = json_dict1['a']['475189']
columns = list(firstplayer.keys())
print(columns)
print(firstplayer['team_id'])


['id', 'goals', 'own_goals', 'shots', 'xG', 'time', 'player_id', 'team_id', 'position', 'player', 'h_a', 'yellow_card', 'red_card', 'roster_in', 'roster_out', 'key_passes', 'assists', 'xA', 'xGChain', 'xGBuildup', 'positionOrder']
83


In [3]:
for url in urls:
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
    scripts = soup.find_all('script')

    rosters_json_string = ''
    for each in scripts:
        if 'rostersData' in each.text:
            rosters_json_string = each.text.strip()
    json_dict = jsonreader(rosters_json_string)
    
    for row in json_dict['h']:
        seasonrosterdata.append(list(json_dict['h'][row].values()))

    for row in json_dict['a']:
        seasonrosterdata.append(list(json_dict['a'][row].values()))



In [4]:
df = pd.DataFrame(seasonrosterdata,columns=columns)

In [5]:
df

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
0,475172,0,0,0,0,90,9676,244,GK,David Raya,...,0,0,0,0,0,0,0,0,0,1
1,475174,0,0,0,0,90,1801,244,DC,Pontus Jansson,...,0,0,0,0,0,0,0,0,0,3
2,475175,0,0,0,0,90,9678,244,DC,Ethan Pinnock,...,0,0,0,0,1,1,0.05214472487568855,0.05214472487568855,0,3
3,475173,0,0,0,0,72,9677,244,DC,Kristoffer Ajer,...,0,0,475184,0,1,0,0.08069062232971191,0.08069062232971191,0,3
4,475176,1,0,3,0.19043469429016113,90,1078,244,DMR,Sergi Canos,...,0,0,0,0,1,0,0.5586957931518555,0.19043469429016113,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,505473,0,0,2,0.15025953948497772,90,3635,88,MC,Bernardo Silva,...,1,0,0,0,1,0,0.021637005731463432,0.09586773812770844,0.02473376877605915,9
551,505475,1,0,2,0.8319949507713318,90,750,88,FWR,Riyad Mahrez,...,0,0,0,0,3,0,0.13943274319171906,0.1589932143688202,0.07423073053359985,14
552,505476,0,0,1,0.072861447930336,65,5543,88,FW,Gabriel Jesus,...,0,0,505478,0,1,0,0.02125939354300499,0.02125939354300499,0,15
553,505477,0,0,1,0.021637005731463432,90,618,88,FWL,Raheem Sterling,...,0,0,0,0,1,0,0.2381884604692459,0.1548079550266266,0.133170947432518,16


In [14]:
# import os
# os.makedirs('/Users/fsa168/Desktop/PythonPractice/SoccerScraper', exist_ok=True)

# df.to_csv('/Users/fsa168/Desktop/PythonPractice/SoccerScraper/full_season_data.csv')

In [29]:
df.head(20)
# filt = (df['team_id'] == 83)

# Arsdf = df.loc[filt, 'player']



,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
0,475172,0,0,0,0,90,9676,83,GK,David Raya,...,0,0,0,0,0,0,0,0,0,1
1,475174,0,0,0,0,90,1801,83,DC,Pontus Jansson,...,0,0,0,0,0,0,0,0,0,3
2,475175,0,0,0,0,90,9678,83,DC,Ethan Pinnock,...,0,0,0,0,1,1,0.05214472487568855,0.05214472487568855,0,3
3,475173,0,0,0,0,72,9677,83,DC,Kristoffer Ajer,...,0,0,475184,0,1,0,0.08069062232971191,0.08069062232971191,0,3
4,475176,1,0,3,0.19043469429016113,90,1078,83,DMR,Sergi Canos,...,0,0,0,0,1,0,0.5586957931518555,0.19043469429016113,0,5
5,475177,0,0,0,0,90,9679,83,DML,Rico Henry,...,0,0,0,0,0,0,0,0,0,6
6,475180,0,0,1,0.05346401408314705,81,9681,83,MC,Frank Onyeka,...,0,0,475183,0,1,0,0.06570620089769363,0.23746681213378906,0.1840028017759323,9
7,475178,1,0,2,1.4452940225601196,90,7083,83,MC,Christian Nørgaard,...,0,0,0,0,0,0,0,0.8865982294082642,0,9
8,475179,0,0,0,0,90,9680,83,MC,Vitaly Janelt,...,0,0,0,0,0,0,0,0.07258376479148865,0.07258376479148865,9
9,475181,0,0,2,0.1989872306585312,87,6552,83,FW,Bryan Mbeumo,...,0,0,475185,0,1,0,0.07258376479148865,0.27157098054885864,0,15


In [28]:
Arsdf

0           David Raya
1       Pontus Jansson
2        Ethan Pinnock
3      Kristoffer Ajer
4          Sergi Canos
            ...       
550     Bernardo Silva
551       Riyad Mahrez
552      Gabriel Jesus
553    Raheem Sterling
554     Ilkay Gündogan
Name: player, Length: 555, dtype: object